---
title: "Final Project"
author: "Suha Akach & Isabelle Perez"
toc: true
number-sections: true
highlight-style: pygments
format: 
  html: 
    code-fold: true
    html-math-method: katex
  pdf: 
    geometry: 
      - top=30mm
      - left=20mm
  docx: default
---


# Import Data & Necessary Packages


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller
from sklearn.metrics import (mean_absolute_error, mean_squared_error,
                                                            r2_score)
from uszipcode import SearchEngine
import seaborn as sns
from pysal.lib import weights
from pysal.model import spreg
import gmplot
import warnings
warnings.filterwarnings("ignore")

In [ ]:
data = pd.read_csv('../data/Drinking_Water_Quality_Distribution_Monitoring_Data_20241116.csv',
                                                                  low_memory = False)                       

# Data Cleaning 


In [ ]:
# rename columns
data.rename(columns = {'Sample Date': 'sample_date', 'Sample Site': 'sample_site', 
              'Sample class': 'sample_class', 'Residual Free Chlorine (mg/L)': 'chlorine',
              'Turbidity (NTU)': 'turbidity', 'Coliform (Quanti-Tray) (MPN /100mL)': 'coliform',
                                      'E.coli(Quanti-Tray) (MPN/100mL)': 'ecoli'}, inplace = True)

In [ ]:
# ensure sample time column is clean
data.dropna(subset = ['Sample Time'], inplace = True)

# standardize format of sample time
data['Sample Time'] = data['Sample Time'].apply(lambda x: x[11:16] if len(x) > 5 else x)

# change to datetime format
data['sample_date'] = pd.to_datetime(data['sample_date'] + ' ' + data['Sample Time'])

# change turbidity to float
data.loc[data['turbidity'] == '<0.10', 'turbidity'] = '0.10'
data['turbidity'] = data['turbidity'].apply(lambda x: float(x))

In [ ]:
# drop unecessary columns and rows
data.drop(columns = ['Sample Number', 'Fluoride (mg/L)', 'Sample Time'], inplace = True)

In [ ]:
# impute non-float values
catval = {'<0.10': float(0.09), '<1': float(0.99), '>200.5': float(200.6)}
data = data.replace(catval)

# change data types to float
data['chlorine'] = data['chlorine'].astype(float)
data['coliform'] = data['coliform'].astype(float)
data['ecoli'] = data['ecoli'].astype(float)

In [ ]:
# check to make sure all values are possible
data = data[data['chlorine'] > 0]
data = data[data['coliform'] > 0]
data = data[data['ecoli'] > 0]

In [ ]:
# only 4 missing values total - drop
data = data.dropna()

data.shape

In [ ]:
# read in data for sampling sites
sites = pd.read_csv('../data/sampling_sites_extended.csv')

# use search engine to find demographic data
search = SearchEngine() 

zipdata = [] 

for zipcode in sites['ZIP Code']: 
  info = search.by_zipcode(zipcode)

  if bool(info) == True: 
    zipdata.append({
      'ZIP Code': zipcode,
      'housing_units': info.housing_units / 1000,
      'occupied_housing_units': info.occupied_housing_units / 1000, 
      'median_home_value': info.median_home_value / 1000,
      'median_household_income': info.median_household_income / 1000
    })

# add demographic data to sampling sites
zipdata = pd.DataFrame(zipdata) 
sites = pd.merge(sites, zipdata, how = 'inner', on = 'ZIP Code')
sites = sites.drop_duplicates()

# merge with location based information
sites.rename(columns = {'Sample Site': 'sample_site'}, inplace = True)

data = pd.merge(data, sites, on = 'sample_site')

In [ ]:
# sort data 
data = data.set_index('sample_date')
data = data.sort_values(by = 'sample_date')

# prepare for modelling
data = data.drop(columns = ['OBJECTID', 'sample_class', 
                            'Sample Station (SS) - Location Description',
                            'X - Coordinate', 'Y - Coordinate',
                            'City or Placename'])

# scale data
data = (data - data.mean()) / data.std()

## Borough information

In [ ]:
# save cleaned and merged data to new file
data.to_csv('../data/data_merged.csv', index=False)

# lowercase column names
data.columns = data.columns.str.lower()

# use uszipcode to find borough from longitude, latitude, and zip code
search = SearchEngine()

# function to get borough from longitude, latitude, and zip code
def get_borough(row):
    try:
        # Use ZIP Code and coordinates to get borough information
        info = search.by_coordinates(row['latitude'], row['longitude'], radius=5)
        if info:
            return info[0].major_city  # This returns the 'borough' or major city name (if available)
        else:
            return None  # If no info is found, return None
    except Exception as e:
        print(f"Error retrieving borough for row {row['sample_site']}: {e}")
        return None

# apply the function to each row of the data
data['borough'] = data.apply(get_borough, axis=1)

# save the final merged dataset to a new file
data.to_csv('../data/data_merged_with_borough.csv', index=False)

# Modelling


In [ ]:
def check_stationarity(series):
    '''Check stationarity of column using Augmented Dickey-Fuller test'''
    result = adfuller(series)
    print(f'ADF Statistic: {result[0]}')
    print(f'p-value: {result[1]}')
    if result[1] <= 0.05:
        print('The series is stationary.')
        print()
    else:
        print('The series is not stationary.')
        print()

# check stationarities
for col in ['chlorine', 'turbidity', 'coliform', 'ecoli']:
    print(f'Checking stationarity for {col}')
    check_stationarity(data[col])

In [ ]:
# split into training and testing
train_size = int(len(data) * 0.7)

data_train = data[:train_size]
data_test = data[train_size:]

In [ ]:
model = VAR(data_train)

In [ ]:
lag_results = {}

# iterate through range of possible lags
for i in range(1, 31):
  result = model.fit(i)
  lag_results[i] = {
    'AIC': result.aic,
    'BIC': result.bic,
    'HQIC': result.hqic,
    }

# find best lag by metric
best_aic = min(lag_results, key = lambda x: lag_results[x]['AIC'])
best_bic = min(lag_results, key = lambda x: lag_results[x]['BIC'])
best_hqic = min(lag_results, key = lambda x: lag_results[x]['HQIC'])

print(f'Best lag by AIC: {best_aic}')
print(f'Best lag by BIC: {best_bic}')
print(f'Best lag by HQIC: {best_hqic}')

In [ ]:
lags = list(lag_results.keys())
aic_values = [lag_results[lag]['AIC'] for lag in lags]
bic_values = [lag_results[lag]['BIC'] for lag in lags]
hqic_values = [lag_results[lag]['HQIC'] for lag in lags]

# plot AIC, BIC, and HQIC for range of lags
plt.plot(lags, aic_values, label='AIC', marker='o')
plt.plot(lags, bic_values, label='BIC', marker='o')
plt.plot(lags, hqic_values, label='HQIC', marker='o')
plt.xlabel('Lag Order')
plt.ylabel('Information Criterion')
plt.title('Lag Order Selection (Custom Range)')
plt.legend()
plt.show()

In [ ]:
# fit model
results = model.fit(maxlags = 27)

# create predictions
forecast_input = data_train.values[-results.k_ar:]
forecast = results.forecast(forecast_input, steps = len(data_test))
forecast_df = pd.DataFrame(forecast, index = data_test.index, 
                                        columns = data_test.columns)

In [ ]:
mae = mean_absolute_error(data_test, forecast_df)
rmse = mean_squared_error(data_test, forecast_df, squared = False)

print(f'MAE: {mae}')
print(f'RMSE: {rmse}')

In [ ]:
for col in data_test.columns:
  actual = data_test[col]
  pred = forecast_df[col]

  print(f'R-squared for {col} on test set: {r2_score(actual, pred):.4f}')

In [ ]:
forecast_df
#data_test


#print(results.test_serial_correlation())
print(results.summary())